Introduction to Pygenprop
=========================
An python library for interactive programatic usage of Genome Properties
------------------------------------------------------------------------

InterProScan files used in this tutorial can be found at:
- https://raw.githubusercontent.com/Micromeda/pygenprop/master/docs/source/_static/tutorial/E_coli_K12.tsv
- https://raw.githubusercontent.com/Micromeda/pygenprop/master/docs/source/_static/tutorial/E_coli_O157_H7.tsv

In [83]:
import requests
from io import StringIO
from pygenprop.results import GenomePropertiesResults
from pygenprop.database_file_parser import parse_genome_properties_flat_file
from pygenprop.assignment_file_parser import parse_interproscan_file, parse_genome_property_longform_file

In [84]:
# The Genome Properties is a flat file database that can be fount on Github.
# The latest release of the database can be found at the following URL.

genome_properties_database_url = 'https://raw.githubusercontent.com/ebi-pf-team/genome-properties/master/flatfiles/genomeProperties.txt'

# For this tutorial we will stream the file directly into the Jupyter notebook. Alternativly, 
# one could be downloaded the file with the unix wget or curl commands.

with requests.Session() as current_download:
    response = current_download.get(genome_properties_database_url, stream=True)
    tree = parse_genome_properties_flat_file(StringIO(response.text))

In [85]:
# There are 1286 properties in the Genome Properties tree.
len(tree)

1286

In [86]:
# Find all properties of type "GUILD".
for genome_property in tree:
    if genome_property.type == 'GUILD':
        print(genome_property.name)

Coenzyme F420 utilization
CRISPR region
Reduction of oxidized methionine
Phage: major features
Resistance to Reactive Oxygen Species (ROS)
tRNA aminoacylation
Toxin-antitoxin system, type II
Protein-coding palindromic elements
Flagellar components of unknown function
Bacillithiol utilization
Toxin-antitoxin system, type I
Toxin-antitoxin system, type III
Abortive infection proteins
Energy-coupling factor transporters
Initiator caspases of the apoptosis extrinsic pathway
Executor caspases of apoptosis


In [87]:
# Parse InterProScan files
with open('E_coli_K12.tsv') as ipr5_file_one:
    assignment_cache_1 = parse_interproscan_file(ipr5_file_one)

In [88]:
with open('E_coli_O157_H7.tsv') as ipr5_file_two:
    assignment_cache_2 = parse_interproscan_file(ipr5_file_two)

In [89]:
# Create results comparison object
results = GenomePropertiesResults(assignment_cache_1, assignment_cache_2, properties_tree=tree)

In [90]:
# Get property by identifier
virulence = tree['GenProp0074']

In [91]:
virulence

GenProp0074, Type: CATEGORY, Name: Virulence, Thresh: 0, References: False, Databases: False, Steps: True, Parents: True, Children: True, Public: False

In [92]:
# Iterate to get the identifiers of child properties of virulence
types_of_vir = [genprop.id for genprop in virulence.children]

In [93]:
# The property results property is used to compare two property assignments between samples.
results.property_results

,E_coli_K12,E_coli_O157_H7
Property_Identifier,,
GenProp0724,NO,NO
GenProp0757,YES,YES
GenProp0809,NO,NO
GenProp0853,NO,NO
GenProp0861,NO,NO
GenProp0901,NO,NO
GenProp0919,NO,NO
GenProp0920,NO,NO
GenProp0921,NO,NO


In [94]:
# The step results property is used to compare two step assignments between samples.
results.step_results

E_coli_K12 E_coli_O157_H7
Property_Identifier Step_Number                          
GenProp0724         1                   NO             NO
                    2                   NO             NO
                    3                   NO             NO
                    4                  YES            YES
                    5                  YES            YES
                    6                   NO             NO
                    7                  YES            YES
                    8                   NO             NO
GenProp0077         2                   NO             NO
                    3                  YES            YES
                    4                   NO             NO
                    5                   NO             NO
                    6                   NO             NO
                    7                   NO             NO
                    8                   NO             NO
                    9                   NO             NO
                    10                  NO             NO
                    11                  NO             NO
                    12                  NO             NO
                    13                  NO             NO
                    14                  NO             NO
                    15                  NO             NO
                    16                  NO             NO
                    17                  NO             NO
                    18                  NO             NO
                    19                  NO             NO
                    20                  NO             NO
                    21                  NO             NO
                    22                  NO             NO
                    23                  NO             NO
...                                    ...            ...
GenProp2095         6                   NO             NO
                    7                   NO             NO
                    8                   NO             NO
                    9                   NO             NO
GenProp2097         1                   NO             NO
                    2                   NO             NO
                    3                   NO             NO
                    4                   NO             NO
GenProp2096         1                   NO             NO
                    2                   NO             NO
                    4                   NO             NO
                    5                   NO             NO
                    6                   NO             NO
GenProp2098         1                   NO             NO
                    2                   NO             NO
                    4                   NO             NO
                    5                   NO             NO
                    6                   NO             NO
                    7                   NO             NO
GenProp2099         1                   NO             NO
                    2                   NO             NO
                    3                  YES            YES
                    4                   NO             NO
                    5                   NO             NO
                    6                   NO             NO
                    7                   NO             NO
                    8                   NO             NO
                    9                   NO             NO
                    10                  NO             NO
                    11                  NO             NO

[6525 rows x 2 columns]

In [95]:
# Get properties with differing assignments
results.differing_property_results

,E_coli_K12,E_coli_O157_H7
Property_Identifier,,
GenProp0111,YES,PARTIAL
GenProp1032,YES,PARTIAL
GenProp0183,YES,PARTIAL
GenProp1695,NO,PARTIAL
GenProp1331,PARTIAL,YES
GenProp1388,NO,YES
GenProp0051,NO,YES
GenProp0232,PARTIAL,YES
GenProp0236,PARTIAL,YES


In [96]:
# Get property assignments for virulence properties
results.get_results(*types_of_vir, steps=False)

,E_coli_K12,E_coli_O157_H7
Property_Identifier,,
GenProp0052,NO,PARTIAL
GenProp0648,YES,YES
GenProp0707,NO,NO


In [97]:
# Get step assignments for virulence properties
results.get_results(*types_of_vir, steps=True)

E_coli_K12 E_coli_O157_H7
Property_Identifier Step_Number                          
GenProp0052         1                   NO             NO
                    2                   NO             NO
                    3                   NO             NO
                    4                   NO             NO
                    5                   NO             NO
                    6                   NO            YES
                    7                   NO             NO
                    8                   NO            YES
                    9                   NO             NO
                    10                 YES            YES
                    11                  NO             NO
                    12                  NO             NO
                    13                  NO            YES
                    14                  NO            YES
                    15                  NO             NO
                    16                  NO            YES
                    17                  NO             NO
                    18                  NO             NO
                    19                  NO            YES
                    21                  NO             NO
                    22                 YES            YES
                    24                  NO            YES
                    25                  NO             NO
                    26                  NO            YES
                    27                  NO            YES
                    28                  NO            YES
                    29                  NO             NO
                    30                  NO             NO
                    31                  NO            YES
                    32                  NO            YES
                    33                  NO             NO
                    34                  NO             NO
                    35                  NO            YES
                    36                  NO             NO
                    37                  NO            YES
                    38                  NO            YES
                    39                  NO            YES
                    40                  NO            YES
                    41                  NO            YES
                    42                  NO            YES
                    43                  NO             NO
                    44                  NO             NO
GenProp0648         1                  YES            YES
                    2                  YES            YES
                    3                  YES            YES
                    4                  YES            YES
                    5                  YES            YES
                    6                  YES            YES
                    7                  YES            YES
GenProp0707         1                   NO             NO
                    2                   NO             NO
                    3                   NO             NO
                    4                   NO             NO
                    5                   NO             NO
                    6                   NO             NO

In [98]:
# Get counts of virulence properties assigned YES, NO, and PARTIAL per organism
results.get_results_summary(*types_of_vir, steps=False, normalize=False)

,E_coli_K12,E_coli_O157_H7
NO,2.0,1
PARTIAL,0.0,1
YES,1.0,1


In [99]:
# Get counts of virulence steps assigned YES, NO, and PARTIAL per organism
results.get_results_summary(*types_of_vir, steps=True, normalize=False)

,E_coli_K12,E_coli_O157_H7
NO,46,27
YES,9,28


In [100]:
# Get percentages of virulence steps assigned YES, NO, and PARTIAL per organism
results.get_results_summary(*types_of_vir, steps=True, normalize=True)

,E_coli_K12,E_coli_O157_H7
NO,83.636364,49.090909
YES,16.363636,50.909091
